In [18]:
import pandas as pd
df = pd.read_csv('../data/NIST_database_onlyH_6TypeEq_pos_match_max_fg_other.csv')
df.head()

,Metal ion,Ligand,Formula,Ligand class,Equilibrium,Temperature (C),Ionic strength,Value type,Value,Note,...,PyridineCount,MonoPhosphateCount,ImidazoleNHCount,CyclicAmineCount,Ligand_class_id,molecular_weight,HBD,HBA,logP,TPSA
0,H⁺,Aminoacetic acid (Glycine),C2H5N1O2,Amino Acids,[HL]/[L][H],25,0.10,Log K,9.57,NaN,...,0,0,0,0,0,75.067,2,2,-0.9703,63.32
1,H⁺,Aminoacetic acid (Glycine),C2H5N1O2,Amino Acids,[HL]/[L][H],37,0.15,Log K,9.25,NaN,...,0,0,0,0,0,75.067,2,2,-0.9703,63.32
2,H⁺,Aminoacetic acid (Glycine),C2H5N1O2,Amino Acids,[HL]/[L][H],25,0.50,Log K,9.54,NaN,...,0,0,0,0,0,75.067,2,2,-0.9703,63.32
3,H⁺,Aminoacetic acid (Glycine),C2H5N1O2,Amino Acids,[HL]/[L][H],25,0.70,Log K,9.59,NaN,...,0,0,0,0,0,75.067,2,2,-0.9703,63.32
4,H⁺,Aminoacetic acid (Glycine),C2H5N1O2,Amino Acids,[HL]/[L][H],25,1.00,Log K,9.66,NaN,...,0,0,0,0,0,75.067,2,2,-0.9703,63.32


In [20]:
df = pd.DataFrame(df["SMILES"].unique(), columns=["SMILES"])
with open("../data/NIST_database_smiles_unique.txt", "w") as f:
    for smiles in df["SMILES"]:
        f.write(f"{smiles}\n")

In [3]:
df1 = df[df['max_eq_num'] == 1]
df2 = df[df['max_eq_num'] == 2]
df3 = df[df['max_eq_num'] == 3]
df4 = df[df['max_eq_num'] == 4]
df5 = df[df['max_eq_num'] == 5]
df6 = df[df['max_eq_num'] == 6]

df1.to_csv("../data/NIST_database_diss_1.csv", index=False)
df2.to_csv("../data/NIST_database_diss_2.csv", index=False)
df3.to_csv("../data/NIST_database_diss_3.csv", index=False)
df4.to_csv("../data/NIST_database_diss_4.csv", index=False)
df5.to_csv("../data/NIST_database_diss_5.csv", index=False)
df6.to_csv("../data/NIST_database_diss_6.csv", index=False)



In [5]:
df = df['SMILES']
df.to_csv("../data/NIST_database_smiles.csv", index=False)

---

In [9]:
import pandas as pd
df_diss = pd.read_csv("../data/nist_pka_data.csv")
df_diss.head()

,SMILES,pKa,Dissociable_Atoms,Functional_Group,Name
0,NCC(O)=O,9.57,0:2,amine_primary:carboxylic_acid,Aminoacetic acid (Glycine)
1,NCC(O)=O,9.25,0:2,amine_primary:carboxylic_acid,Aminoacetic acid (Glycine)
2,NCC(O)=O,9.54,0:2,amine_primary:carboxylic_acid,Aminoacetic acid (Glycine)
3,NCC(O)=O,9.59,0:2,amine_primary:carboxylic_acid,Aminoacetic acid (Glycine)
4,NCC(O)=O,9.66,0:2,amine_primary:carboxylic_acid,Aminoacetic acid (Glycine)


In [15]:
# # 計算Dissociable_Atoms為空的記錄數量
# empty_dissociable_atoms = df_diss[(df_diss['Dissociable_Atoms'] == 0)].shape[0]
# print(f"Dissociable_Atoms為空的記錄數量: {empty_dissociable_atoms}")
# 計算Functional_Group為"unknown"的記錄數量
unknown_functional_group = df_diss[df_diss['Functional_Group'] == 'unknown'].shape[0]
print(f"Functional_Group為'unknown'的記錄數量: {unknown_functional_group}")


Functional_Group為'unknown'的記錄數量: 138


In [21]:
# 讀取錯誤日誌文件
with open("../data/nist_pka_data_errors.log", "r") as f:
    error_lines = f.readlines()

# 提取所有包含"無法識別解離位點:"的行
error_smiles_lines = [line.strip() for line in error_lines if "無法識別解離位點:" in line]

# 從這些行中提取SMILES字串
error_smiles = [line.split(": ")[1] for line in error_smiles_lines]

# 獲取獨立的SMILES字串（去重）
unique_error_smiles = list(set(error_smiles))

# 打印獨立的SMILES字串數量和前10個例子
print(f"獨立的SMILES字串總數: {len(unique_error_smiles)}")

with open("../data/nist_pka_data_errors_unique.log", "w") as f:
    for smiles in unique_error_smiles:
        f.write(f"{smiles}\n")

獨立的SMILES字串總數: 15


In [22]:
df_ori = pd.read_csv('../data/NIST_database_onlyH_6TypeEq_pos_match_max_fg_other.csv')

# 讀取錯誤SMILES列表
with open("../data/nist_pka_data_errors_unique.log", "r") as f:
    error_smiles_list = [line.strip() for line in f.readlines()]

# 在原始數據中查詢這些SMILES對應的Ligand Class
error_compounds = []
for smiles in error_smiles_list:
    matches = df_ori[df_ori['SMILES'] == smiles]
    if not matches.empty:
        for _, row in matches.iterrows():
            error_compounds.append({
                'Ligand Class': row['Ligand class'] if 'Ligand class' in row else 'Unknown',
                'SMILES': smiles,
                'Ligand': row['Ligand'] if 'Ligand' in row else 'Unknown'
            })
    else:
        error_compounds.append({
            'Ligand Class': 'Not Found',
            'SMILES': smiles,
            'Ligand': 'Not Found'
        })

# 將結果保存到新檔案
error_df = pd.DataFrame(error_compounds)
error_df = error_df.drop_duplicates(subset=['SMILES'])
error_df = error_df.sort_values(by='Ligand Class')
error_df.to_csv('../data/nist_pka_errors_with_class.csv', index=False)
print(f"已將{len(error_compounds)}個錯誤SMILES的Ligand Class資訊保存到檔案")


已將24個錯誤SMILES的Ligand Class資訊保存到檔案


---

In [23]:
# nist_pka_data
nist_pka_data = pd.read_csv("../data/nist_pka_data.csv")
nist_pka_data = nist_pka_data[nist_pka_data['Functional_Group'] != 'unknown']
nist_pka_data.to_csv("../data/nist_pka_data_clean.csv", index=False)

In [5]:
import pandas as pd
# 讀取包含解離順序的數據
df_with_order = pd.read_csv("../data/nist_pka_data_with_order.csv")

# 通過檢查Dissociable_Atoms中的冒號數量來判斷解離原子數量
# 篩選出不同數量解離原子的數據
binary_df = df_with_order[df_with_order['Dissociable_Atoms'].str.count(':') == 1]
print(f"\n篩選出{len(binary_df)}行只有兩個解離原子的數據")
binary_df.to_csv("../data/nist_pka_data_binary_dissociation.csv", index=False)
print(f"已將二元解離數據保存至 ../data/nist_pka_data_binary_dissociation.csv")

# 生成1~6個解離原子的數據檔案
for i in range(1, 7):
    # 對於1個解離原子，冒號數量為0；對於n個解離原子，冒號數量為n-1
    colon_count = i - 1
    filtered_df = df_with_order[df_with_order['Dissociable_Atoms'].str.count(':') == colon_count]
    output_path = f"../data/nist_pka_data_{i}_dissociation.csv"
    filtered_df.to_csv(output_path, index=False)
    print(f"已將{i}個解離原子的數據{len(filtered_df)}行保存至 {output_path}")



篩選出2923行只有兩個解離原子的數據
已將二元解離數據保存至 ../data/nist_pka_data_binary_dissociation.csv
已將1個解離原子的數據1980行保存至 ../data/nist_pka_data_1_dissociation.csv
已將2個解離原子的數據2923行保存至 ../data/nist_pka_data_2_dissociation.csv
已將3個解離原子的數據903行保存至 ../data/nist_pka_data_3_dissociation.csv
已將4個解離原子的數據712行保存至 ../data/nist_pka_data_4_dissociation.csv
已將5個解離原子的數據108行保存至 ../data/nist_pka_data_5_dissociation.csv
已將6個解離原子的數據92行保存至 ../data/nist_pka_data_6_dissociation.csv
